In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
RANDOM_STATE = 42

In [3]:
# downloading data
dataset = load_boston()

X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names

y = dataset.target

In [4]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

print(f'Размер обучающей выборки: {X_train.shape}; тестовых значений: {y_train.shape}')
print(f'Размер тестовой выборки: {X_test.shape}; тестовых значений: {y_test.shape}')

Размер обучающей выборки: (404, 13); тестовых значений: (404,)
Размер тестовой выборки: (102, 13); тестовых значений: (102,)


2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [6]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import r2_score

models = [LinearRegression(), Lasso(), Ridge()]
r2_scores = []

for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    curr_score = r2_score(y_test, y_pred)
    r2_scores.append(curr_score)

for score, model in zip(r2_scores, ['LinearRegression', 'Lasso', 'Ridge']):
    print(f'R2_score для {model}: {score:.4f}')

R2_score для LinearRegression: 0.6968
R2_score для Lasso: 0.6803
R2_score для Ridge: 0.6898


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [7]:
from sklearn.model_selection import GridSearchCV

l1_model, l2_model = Lasso(), Ridge()
params = {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e2, 1e3, 1e4, 1e5]}

GRD_CV_l1 = GridSearchCV(l1_model, params, cv=5, scoring='r2')
GRD_CV_l2 = GridSearchCV(l2_model, params, cv=5, scoring='r2')

# CV проводим на train выборке, чтобы модель не видела test-выборку
GRD_CV_l2.fit(X, y)
GRD_CV_l1.fit(X, y)


print(f'Лучший score для Lasso: {GRD_CV_l1.best_score_:.4f}, лучший параметр alpha: {GRD_CV_l1.best_params_["alpha"]}')
print(f'Лучший score для Ridge: {GRD_CV_l2.best_score_:.4f}, лучший параметр alpha: {GRD_CV_l2.best_params_["alpha"]}')

Лучший score для Lasso: 0.4318, лучший параметр alpha: 1
Лучший score для Ridge: 0.4932, лучший параметр alpha: 100.0


In [8]:
# обучим модель на train, протестируем на test
models_with_alpha = [Lasso(alpha=GRD_CV_l1.best_params_["alpha"]),
          Ridge(alpha=GRD_CV_l2.best_params_["alpha"])]
r2_scores_with_alpha = []

for model in models_with_alpha:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    curr_score = r2_score(y_test, y_pred)
    r2_scores_with_alpha.append(curr_score)

for score_before, score_after, model in zip(r2_scores_with_alpha, r2_scores, ['Lasso', 'Ridge']):
    print(f'R2_score с параметром alpha по умолчанию для {model}: {score_before:.4f}')
    print(f'R2_score с параметром alpha после подбора для {model}: {score_after:.4f}')

R2_score с параметром alpha по умолчанию для Lasso: 0.6803
R2_score с параметром alpha после подбора для Lasso: 0.6968
R2_score с параметром alpha по умолчанию для Ridge: 0.6912
R2_score с параметром alpha после подбора для Ridge: 0.6803


Для модели Lasso с параметром alpha после подбора значение R2 увеличилось, в модели Ridge же R2 наоборот уменьшилось.

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

pipe_with_stdscaler = Pipeline([('scaler', StandardScaler()), ('LinearReg', LinearRegression())])
pipe_with_stdscaler.fit(X_train, y_train)
print(f'R2 для LinearRegression с StandardScaler: {pipe_with_stdscaler.score(X_test, y_test)}')


pipe_with_minmaxscaler = Pipeline([('scaler', MinMaxScaler()), ('LinearReg', LinearRegression())])
pipe_with_minmaxscaler.fit(X_train, y_train)
print(f'R2 для LinearRegression с MinMaxScaler: {pipe_with_minmaxscaler.score(X_test, y_test)}')

R2 для LinearRegression с StandardScaler: 0.6968039999753707
R2 для LinearRegression с MinMaxScaler: 0.6968039999753716


Результат увеличился, R2 достиг значений как у Lasso. При этом scaler'ы показывают близкий результат.

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [10]:
pipe_lasso_mmscaler = Pipeline([('scaler', MinMaxScaler()), ('lasso', Lasso())])
pipe_ridge_mmscaler = Pipeline([('scaler', MinMaxScaler()), ('ridge', Ridge())])
params_lasso = {'lasso__alpha': 10 ** np.linspace(-5, 5, 11)}
params_ridge = {'ridge__alpha': 10 ** np.linspace(-5, 5, 11)}

GRD_L1_mmscaler = GridSearchCV(pipe_lasso_mmscaler, params_lasso, cv=5, scoring='r2')
GRD_L1_mmscaler.fit(X, y)
best_l1_alpha = GRD_L1_mmscaler.best_params_['lasso__alpha']

GRD_L2_mmscaler = GridSearchCV(pipe_ridge_mmscaler, params_ridge, cv=5, scoring='r2')
GRD_L2_mmscaler.fit(X, y)
best_l2_alpha = GRD_L2_mmscaler.best_params_['ridge__alpha']

# обучим модель на train, протестируем на test
models_with_alpha = [Lasso(alpha=best_l1_alpha),
                     Ridge(alpha=best_l2_alpha)]
r2_scores_with_alpha = []

for model in models_with_alpha:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    curr_score = r2_score(y_test, y_pred)
    r2_scores_with_alpha.append(curr_score)

for score_after, model in zip(r2_scores_with_alpha, ['Lasso', 'Ridge']):
    print(f'R2_score с параметром подобранным параметром alpha и MinMaxScaler для {model}: {score_after:.4f}')

R2_score с параметром подобранным параметром alpha и MinMaxScaler для Lasso: 0.6959
R2_score с параметром подобранным параметром alpha и MinMaxScaler для Ridge: 0.6898


Качество стало еще лучше, но у LinearRegression без регуляризации всё таки R2 немного больше.
Вообще довольно сильно на результат влияет разбиение на train и test. В этом можно убедиться, изучив результат GridSearchCV:

In [11]:
scores_cv = []
for i in range(5):
    scores_cv.append(GRD_L1_mmscaler.cv_results_[f'split{i}_test_score'][0])

print(f'R2 при разном разбиении на train/test для Lasso ghb alpha={params_lasso["lasso__alpha"][0]}:\n{scores_cv}')

R2 при разном разбиении на train/test для Lasso ghb alpha=1e-05:
[0.6392278898953108, 0.7138862609304375, 0.5870266186274171, 0.07924484550734123, -0.2526663715292563]


Или достаточно посмотреть std R2 при подборе коэффициента регуляризации, std очень большое в данном случае:

In [12]:
GRD_L1_mmscaler.cv_results_['std_test_score'].tolist()

[0.37648553851684025,
 0.37574933850460285,
 0.3685983398728716,
 0.3281721647115124,
 0.29285275190609206,
 0.6816257098927414,
 0.9299864046290232,
 0.9299864046290232,
 0.9299864046290232,
 0.9299864046290232,
 0.9299864046290232]

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [13]:
from sklearn.preprocessing import PolynomialFeatures


pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('pol_feat', PolynomialFeatures(degree=2, include_bias=False)),
                 ('model', LinearRegression())])

pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.7599265018755852

Значение R2 стало еще больше, результат улучшился.

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

Так как выше было показано, что разбиение выборки на train/test при CV сильно влияет на результат, то необходимо перемешивать объекты в группах:

In [14]:
from sklearn.model_selection import KFold

# создадим собственное разбиение с перемешиванием
cv = KFold(n_splits=5, shuffle=True)

# начальный PipeLine
pipe = Pipeline([('Scaler', MinMaxScaler()),
                 ('PolFeatures', PolynomialFeatures(include_bias=False)),
                 ('model', LinearRegression())])

params_grid = [{
    'Scaler': [StandardScaler(), MinMaxScaler()],
    'model': [LinearRegression()],
    'PolFeatures__degree': [2, 3, 4],
},
{
    'Scaler': [StandardScaler(), MinMaxScaler()],
    'model': [Lasso()],
    'PolFeatures__degree': [2, 3, 4],
    'model__alpha': 10 ** np.linspace(-5, 5, 11)
},
{
    'Scaler': [StandardScaler(), MinMaxScaler()],
    'model': [Ridge()],
    'PolFeatures__degree': [2, 3, 4],
    'model__alpha': 10 ** np.linspace(-5, 5, 11)
}]

grd = GridSearchCV(estimator=pipe,
                   param_grid=params_grid,
                   cv=cv,
                   scoring='r2')
grd.fit(X, y);

In [15]:
# пример нескольких значений в сетке GridsearchCV
grid_sample = np.array(grd.cv_results_['params'])
ind = np.random.choice(len(grd.cv_results_['params']), size=3)

grid_sample[ind].tolist()

[{'PolFeatures__degree': 4,
  'Scaler': StandardScaler(),
  'model': Ridge(),
  'model__alpha': 0.0001},
 {'PolFeatures__degree': 4,
  'Scaler': StandardScaler(),
  'model': LinearRegression()},
 {'PolFeatures__degree': 3,
  'Scaler': MinMaxScaler(),
  'model': Lasso(alpha=0.001),
  'model__alpha': 0.1}]

In [16]:
print(f'Лучшая модель и её параметры:\n{grd.best_params_}\nЗначение R2 на CV=5: {grd.best_score_:.4f}')

Лучшая модель и её параметры:
{'PolFeatures__degree': 3, 'Scaler': MinMaxScaler(), 'model': Lasso(alpha=0.001), 'model__alpha': 0.001}
Значение R2 на CV=5: 0.8832


Результат значительно улучшился, благодаря использованию Scaler, PolynomialFeatures и подбору коэффициента регуляризации удалось достичь высокого значения R2.

# Часть 2

http://archive.ics.uci.edu/ml/datasets/Adult

In [17]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

header = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
          'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

data.columns = header
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [18]:
X, y = data.drop(columns='income', axis=1), data[['income']]
y['income'] = y['income'].apply(lambda x: 0 if x == '<=50K' else 1)

X.shape, y.shape

((48842, 14), (48842, 1))

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [19]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       48842 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      48842 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48842 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


Видно, что пропусков в колонках нет.

10. Выберите колонки с числовыми и категориальными переменными.

In [20]:
numeric_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_cols = list(set(X.columns) - set(numeric_cols))

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [21]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = Pipeline(steps=[('OHE', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# пример прохождения всего пайплайна
data_trans = preprocessor.fit_transform(X)
data_trans = pd.DataFrame(data_trans.toarray()).head(5)
data_trans.columns = preprocessor.get_feature_names_out()
data_trans

,num__age,num__fnlwgt,num__education-num,num__capital-gain,num__capital-loss,num__hours-per-week,cat__workclass_?,cat__workclass_Federal-gov,cat__workclass_Local-gov,cat__workclass_Never-worked,...,cat__education_Masters,cat__education_Preschool,cat__education_Prof-school,cat__education_Some-college,cat__relationship_Husband,cat__relationship_Not-in-family,cat__relationship_Other-relative,cat__relationship_Own-child,cat__relationship_Unmarried,cat__relationship_Wife
0,0.301370,0.044131,0.800000,0.02174,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.452055,0.048052,0.800000,0.00000,0.0,0.122449,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.287671,0.137581,0.533333,0.00000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.493151,0.150486,0.400000,0.00000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.150685,0.220635,0.800000,0.00000,0.0,0.397959,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [22]:
from sklearn.metrics import accuracy_score, f1_score

most_common_class = y.value_counts().index[0][0]
y_pred = [most_common_class] * len(y)

print(f'Значение accuracy для "наивного" классификатора: {accuracy_score(y, y_pred):.4f}')
print(f'Значение f1_score для "наивного" классификатора: {f1_score(y, y_pred)}')

Значение accuracy для "наивного" классификатора: 0.7607
Значение f1_score для "наивного" классификатора: 0.0


F1 будет равен 0, так как нет ни одного TP (true positive) и recall = 0. Самый популярный класс - это 0 (<=50K)

13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import cross_validate

models = [LogisticRegression(), SVC(), LinearSVC()]
cv = KFold(n_splits=5, shuffle=True)

for model, model_name in zip(models, ['LogisticRegression', 'SVC', 'LinearSVC']):
    # используем прошлый pipeline из п.11
    pipe = Pipeline([('transformer', preprocessor), ('classifier', model)])
    cv_model = cross_validate(pipe, X, y, cv=cv, scoring=['accuracy', 'f1'])
    score_acc, score_f1 = cv_model['test_accuracy'].mean(), cv_model['test_f1'].mean()
    print(f'Оценка модели {model_name} на тестовых данных:')
    print(f'Accuracy: {score_acc:.4f}\nF1_score: {score_f1:.4f}\n')

Оценка модели LogisticRegression на тестовых данных:
Accuracy: 0.8512
F1_score: 0.6566

Оценка модели SVC на тестовых данных:
Accuracy: 0.8403
F1_score: 0.6208

Оценка модели LinearSVC на тестовых данных:
Accuracy: 0.8529
F1_score: 0.6575



14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [78]:
from sklearn.impute import SimpleImputer

numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = Pipeline(steps=[('fill_missing', SimpleImputer(missing_values='?', strategy='most_frequent')),
                                          ('OHE', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

15. Посчитайте cross_val_score на новых данных. Напишите, удалось ли улучшить результат.

In [79]:
models = [LogisticRegression(), SVC(), LinearSVC()]
cv = KFold(n_splits=5, shuffle=True)

for model, model_name in zip(models, ['LogisticRegression', 'SVC', 'LinearSVC']):
    # используем прошлый pipeline из п.14
    pipe = Pipeline([('transformer', preprocessor), ('classifier', model)])
    cv_model = cross_validate(pipe, X, y, cv=cv, scoring=['accuracy', 'f1'])
    score_acc, score_f1 = cv_model['test_accuracy'].mean(), cv_model['test_f1'].mean()
    print(f'Оценка модели {model_name} на тестовых данных:')
    print(f'Accuracy: {score_acc:.4f}\nF1_score: {score_f1:.4f}\n')

Оценка модели LogisticRegression на тестовых данных:
Accuracy: 0.8509
F1_score: 0.6551

Оценка модели SVC на тестовых данных:
Accuracy: 0.8394
F1_score: 0.6165

Оценка модели LinearSVC на тестовых данных:
Accuracy: 0.8513
F1_score: 0.6521



Качество моделей не изменилось, даже где-то стало немного хуже.

16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [85]:
data.replace('?', np.nan).dropna(how='any')

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,NaN,321403,HS-grad,9,Widowed,NaN,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [87]:
numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = Pipeline(steps=[('OHE', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# удалим строки с хотя бы одним значением '?' в одном из признаков
X_removed, y_removed = data.replace('?', np.nan).dropna(how='any') \
                           .drop(columns='income', axis=1), data.replace('?', np.nan).dropna(how='any')[['income']]
y_removed['income'] = y_removed['income'].apply(lambda x: 0 if x == '<=50K' else 1)


models = [LogisticRegression(), SVC(), LinearSVC()]
cv = KFold(n_splits=5, shuffle=True)

for model, model_name in zip(models, ['LogisticRegression', 'SVC', 'LinearSVC']):
    # используем прошлый pipeline из п.14
    pipe = Pipeline([('transformer', preprocessor), ('classifier', model)])
    cv_model = cross_validate(pipe, X_removed, y_removed, cv=cv, scoring=['accuracy', 'f1'])
    score_acc, score_f1 = cv_model['test_accuracy'].mean(), cv_model['test_f1'].mean()
    print(f'Оценка модели {model_name} на тестовых данных:')
    print(f'Accuracy: {score_acc:.4f}\nF1_score: {score_f1:.4f}\n')

Оценка модели LogisticRegression на тестовых данных:
Accuracy: 0.8472
F1_score: 0.6605

Оценка модели SVC на тестовых данных:
Accuracy: 0.8356
F1_score: 0.6262

Оценка модели LinearSVC на тестовых данных:
Accuracy: 0.8486
F1_score: 0.6618



После удаления элементов с '?' в качестве признака, метрика f1 немного повысилась.

 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [89]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

models = [RandomForestClassifier(), GradientBoostingClassifier()]
cv = KFold(n_splits=5, shuffle=True)

for model, model_name in zip(models, ['RandomForestClassifier', 'GradientBoostingClassifier']):
    # используем прошлый pipeline из п.14
    pipe = Pipeline([('transformer', preprocessor), ('classifier', model)])
    cv_model = cross_validate(pipe, X, y, cv=cv, scoring=['accuracy', 'f1'])
    score_acc, score_f1 = cv_model['test_accuracy'].mean(), cv_model['test_f1'].mean()
    print(f'Оценка модели {model_name} на тестовых данных:')
    print(f'Accuracy: {score_acc:.4f}\nF1_score: {score_f1:.4f}\n')

Оценка модели RandomForestClassifier на тестовых данных:
Accuracy: 0.8537
F1_score: 0.6703

Оценка модели GradientBoostingClassifier на тестовых данных:
Accuracy: 0.8677
F1_score: 0.6860



F1_score стал еще больше при применении новых алгоритмов.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [102]:
from sklearn.preprocessing import OrdinalEncoder

numeric_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = Pipeline(steps=[('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

result_pipe = Pipeline([('transforms', preprocessor), ('model', LogisticRegression())])

params_grid = {
    'transforms__num__scaler': [MinMaxScaler(), StandardScaler()],
    'transforms__cat__encoder': [OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1),
                                OneHotEncoder(handle_unknown='ignore')],
    'model': [LogisticRegression(), RandomForestClassifier(), LinearSVC(), GradientBoostingClassifier()]
}

cv = KFold(n_splits=5, shuffle=True)

# для нашей задачи метрика accuracy не очень информативна, необходимо максимизировать метрику f1.
grd = GridSearchCV(result_pipe,
                   param_grid=params_grid,
                   cv=cv,
                   scoring='f1')
grd.fit(X, y);

In [103]:
print(f'Лучшая модель и её параметры:\n{grd.best_params_}\nЗначение F1 на CV=5: {grd.best_score_:.4f}')

Лучшая модель и её параметры:
{'model': GradientBoostingClassifier(), 'transforms__cat__encoder': OneHotEncoder(handle_unknown='ignore'), 'transforms__num__scaler': MinMaxScaler()}
Значение F1 на CV=5: 0.6856
